In [3]:
import pandas as pd

AA_ALPHABET = [
    "A",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "K",
    "L",
    "M",
    "N",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "V",
    "W",
    "Y",
]

In [ ]:
with open(
    "data/scope40_prot-md-pssm-2025-03-05-17-43-47-full-dataset_concatenated.tsv",
    "r",
) as f:
    data = f.read()

profiles = data.split("Query profile of sequence ")

if profiles[0].strip() == "":
    profiles = profiles[1:]

profile_dict = {}
for profile in profiles:
    if not profile.strip():
        continue

    lines = profile.strip().split("\n")
    seq_id = lines[0]

    aa_cols = lines[1].split()

    matrix_data = []
    for line in lines[2:]:
        if line.strip():
            matrix_data.append([float(x) for x in line.split()])

    profile_df = pd.DataFrame(matrix_data, columns=aa_cols)
    profile_dict[seq_id] = profile_df

print(len(profile_dict))


In [11]:
seq_dict = {}
with open("data/scope40_sequences_3Di.fasta") as f:
    current_id = None
    current_seq = []
    for line in f:
        line = line.strip()
        if line.startswith(">"):
            if current_id is not None:
                seq_dict[current_id] = "".join(current_seq)
            current_id = line[1:]
            current_seq = []
        else:
            current_seq.append(line)
    if current_id is not None:
        seq_dict[current_id] = "".join(current_seq)


In [ ]:
def get_argmax_aa(profile_matrix):
    """Get the amino acid with highest probability for each position.

    Args:
        profile_matrix: DataFrame containing position-specific scoring matrix

    Returns:
        List of amino acids with highest probability at each position
    """
    # Get argmax index for each row
    argmax_indices = list(profile_matrix.values.argmax(axis=1))

    # Map indices to amino acids using AA_ALPHABET
    argmax_aa = "".join([AA_ALPHABET[idx] for idx in argmax_indices])

    return argmax_aa


for name, profile in list(profile_dict.items()):
    print(name)
    print(get_argmax_aa(profile))
    print(seq_dict[name])


In [16]:
with open("data/scope40_sequences_3Di_argmax.fasta", "w") as f:
    for name, profile in profile_dict.items():
        f.write(f">{name}\n")
        argmax_seq = get_argmax_aa(profile)
        f.write(f"{argmax_seq}\n")
